In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os 
import pandas as pd 
import sys 
from tqdm import tqdm 


In [8]:
sys.path.append("../")

In [19]:
datadir = "../output/evaluation/killable_check"
projects = ['Lang', 'Math', 'Time', 'Compress', 'Cli', 'Codec', #'Gson', 
            'Closure', 'Collections', 'Jsoup', 'Csv', #'Mockito', 
            'JxPath', 'JacksonXml', 'JacksonCore']#, 'JacksonDatabind']

In [7]:
len(projects)

14

### Checking the statistics of candidates of latent mutants killable at the point of introduciton 

* all latent mutants 

* sampling criteria: between numsel and percentage, percentage has the priority, as long as it would be larger than min_numsel. So, min_numSel, perc, numSel
    - min_numsel: the minium number of selected mutants. Currently, 10 
    - numsel: the number of selected mutants 
    - percentage: the ratio of selected mutants. If percentage * # numbers < numsel, then we select percentage * # numbers 

* candidate file: 
    - {project}.target_latentmuts.pkl: contains all candidates 
    - {project}.{col_name}_perc{perc}.pkl: contains perc% of randomly sampled candidates


In [10]:
# set defects4j environment 
os.environ['D4J_HOME'] = '/Users/jeongju.sohn/workdir/tools/defects4j'

In [16]:
from check_killable_atIntro import old_outputdir, get_latent_mut_data

In [20]:
outputdir = "../output/evaluation"
num_stats = []
latent_mut_df_proj = {}
for project in projects:
    # for 
    mut_propa_stat_debt_file = os.path.join(
        outputdir, 
        f"processed/{project}.indv_mut_propagation_status_and_debt.csv") 
    df = pd.read_csv(mut_propa_stat_debt_file)
    df = df.loc[df.status == 'latent'] 
    latent_mut_df = get_latent_mut_data(outputdir, project)
    latent_mut_df_proj[project] = latent_mut_df
    if latent_mut_df is not None:
        n_latent_mut_df = latent_mut_df.shape[0]
    else:
        n_latent_mut_df = 0 
    num_stats.append([project, df.shape[0], n_latent_mut_df])
num_stats = pd.DataFrame(num_stats, columns = ['project', 'org_num', 'appr_num'])

2024-01-31 10:53:07,198 Start collecting latent mutant information
2024-01-31 10:53:07,292 latent mutant data and respective failing test data collected
2024-01-31 10:53:07,297 latent mutant data and respective failing test data merged
2024-01-31 10:53:07,308 Start collecting latent mutant information
2024-01-31 10:53:07,394 latent mutant data and respective failing test data collected
2024-01-31 10:53:07,401 latent mutant data and respective failing test data merged
2024-01-31 10:53:07,406 Start collecting latent mutant information
2024-01-31 10:53:07,424 latent mutant data and respective failing test data collected
2024-01-31 10:53:07,429 latent mutant data and respective failing test data merged
2024-01-31 10:53:07,440 Start collecting latent mutant information
2024-01-31 10:53:07,501 latent mutant data and respective failing test data collected
2024-01-31 10:53:07,506 latent mutant data and respective failing test data merged
2024-01-31 10:53:07,511 Start collecting latent mutant i

In [24]:
num_stats.loc[num_stats.org_num > 0]

,project,org_num,appr_num
0,Lang,78,71
1,Math,163,144
2,Time,9,9
3,Compress,138,136
4,Cli,15,13
5,Codec,24,24
6,Closure,60,58
8,Jsoup,81,81
9,Csv,6,6
12,JacksonCore,61,60


### Compare with the candidates 

In [32]:
candidates, sampled = {}, {}
n_targets, n_sampled = [], []
for project in projects:
    target_datafile = os.path.join(datadir, f"{project}.target_latentmuts.pkl")
    if not os.path.exists(target_datafile):
        n_targets.append(0)
        n_sampled.append(0)
        continue 
    target_df = pd.read_pickle(target_datafile)
    sampled_datafile = os.path.join(datadir, f"{project}.debt_time_perc50.pkl")
    sampled_df = pd.read_pickle(sampled_datafile)
    candidates[project] = target_df
    sampled[project] = sampled_df
    n_targets.append(target_df.shape[0])
    n_sampled.append(sampled_df.shape[0])


In [33]:
# update num stats
num_stats['cand_num'] = n_targets
num_stats['sampled_num'] = n_sampled

In [35]:
print(num_stats)

        project  org_num  appr_num  cand_num  sampled_num
0          Lang       78        71        29           15
1          Math      163       144        29           20
2          Time        9         9         9            9
3      Compress      138       136        43           20
4           Cli       15        13         4            4
5         Codec       24        24         6            6
6       Closure       60        58        16           16
7   Collections        0         0         0            0
8         Jsoup       81        81        30           20
9           Csv        6         6         6            6
10       JxPath        0         0         0            0
11   JacksonXml        0         0         0            0
12  JacksonCore       61        60         3            3


In [37]:
num_stats.loc[num_stats.cand_num > 0]

,project,org_num,appr_num,cand_num,sampled_num
0,Lang,78,71,29,15
1,Math,163,144,29,20
2,Time,9,9,9,9
3,Compress,138,136,43,20
4,Cli,15,13,4,4
5,Codec,24,24,6,6
6,Closure,60,58,16,16
8,Jsoup,81,81,30,20
9,Csv,6,6,6,6
12,JacksonCore,61,60,3,3


In [41]:
_n_df = num_stats.loc[num_stats.cand_num > 0]
print (_n_df.cand_num/_n_df.appr_num)
print ()
print (_n_df.cand_num/_n_df.org_num)

0     0.408451
1     0.201389
2     1.000000
3     0.316176
4     0.307692
5     0.250000
6     0.275862
8     0.370370
9     1.000000
12    0.050000
dtype: float64

0     0.371795
1     0.177914
2     1.000000
3     0.311594
4     0.266667
5     0.250000
6     0.266667
8     0.370370
9     1.000000
12    0.049180
dtype: float64


so far, there are some

In [44]:
candidates['Time'].columns 

Index(['bid', 'mutK', 'status', 'mutOp', 'lno', 'debt_time', 'debt_nc',
       'ftests', 'revealedAt', 'ftesthist', 'ftestpath'],
      dtype='object')

In [53]:
[v[-1] for v in candidates['Time'].iloc[0].ftesthist.values()][-1][-1]

True

In [51]:
candidates['Time'].iloc[0].ftesthist

{'org.joda.time.TestPartial_Basics#testWith_baseAndArgHaveNoRange': [['8612f9e5',
   'src/test/java/org/joda/time/TestPartial_Basics.java',
   True]]}

In [48]:
candidates['Time'].iloc[0].ftestpath[-1]

{'org.joda.time.TestPartial_Basics#testWith_baseAndArgHaveNoRange': 'src/test/java/org/joda/time/TestPartial_Basics.java'}

In [64]:
def count_newly_introduced_ftests(row:pd.Series):
    cnt = 0
    for vs in row.ftesthist.values():
        cnt += int(vs[-1][-1])
    return cnt/len(row.ftestpath)

In [71]:
for project in projects:
    if project not in candidates.keys(): continue
    cnts = []
    for _, row in candidates[project].iterrows():
        cnt = count_newly_introduced_ftests(row)
        cnts.append(cnt)
    print (f"{project}")
    print (f"\tout of {len(cnts)}, {cnts.count(1)} were newly introduced tests")

Lang
	out of 29, 7 were newly introduced tests
Math
	out of 29, 21 were newly introduced tests
Time
	out of 9, 6 were newly introduced tests
Compress
	out of 43, 25 were newly introduced tests
Cli
	out of 4, 0 were newly introduced tests
Codec
	out of 6, 0 were newly introduced tests
Closure
	out of 16, 10 were newly introduced tests
Jsoup
	out of 30, 23 were newly introduced tests
Csv
	out of 6, 4 were newly introduced tests
JacksonCore
	out of 3, 3 were newly introduced tests


In [87]:
for v in candidates['Lang'].iloc[11].ftesthist.values():
    print ([av[-1] for av in v])

[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]


In [90]:
candidates['Lang'].iloc[0].ftesthist

{'org.apache.commons.lang3.math.NumberUtilsTest#testCreateNumberMagnitude': [['a2de6040',
   'src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java',
   False],
  ['57112a6b',
   'src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java',
   True]]}

In [91]:
candidates['Lang'].iloc[0].ftestpath

{'org.apache.commons.lang3.math.NumberUtilsTest#testCreateNumberMagnitude': 'src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java'}

In [ ]:
for p,candidate in candidates.items():
    candidate.fte
    break 

In [160]:
for project in candidates.keys():
    for _, row in candidates[project].iterrows():
        ftests = row.ftests.split(",")
        cnts = [ftest.count('#') for ftest in ftests]
        assert sum(cnts) == len(ftests)

In [106]:
import utils.parser_utils as parser_utils
import utils.file_utils as file_utils 
import javalang
import contextlib 


In [228]:
content = file_utils.readFile("../workdir/refs/Lang1/src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java")
tree = parser_utils.parse(content)
build_pos_dict, lno_to_node_dict = parser_utils.build_positon_dict(tree)

In [268]:
def get_final_line(node: javalang.tree.Node) -> int:
    # traverse node and find the max line
    max_line = 0
    for path, child in node.filter(javalang.tree.Node):
        with contextlib.suppress(TypeError):
            max_line = max(max_line, child.position[0])
    return max_line

def build_position_dict(tree: javalang.tree.CompilationUnit):
    package_name = None if tree.package is None else tree.package.name
    ret = {"package": package_name, "classes": []}
    for path, node in tree.filter(javalang.tree.ClassDeclaration):
        classDeclNode: javalang.tree.ClassDeclaration = node
        class_name = classDeclNode.name
        class_pos = (
            classDeclNode.position[0],
            get_final_line(classDeclNode),
        )
        class_dict = {"name": class_name, "pos": class_pos, "methods": []}
        for method in classDeclNode.methods:
            method_dict = {
                "name": method.name,
                "pos": (method.position[0], get_final_line(method)),
                "paramtypes": [p.type.name for p in method.parameters],
            }
            class_dict["methods"].append(method_dict)
        class_dict["inner"] = None
        for prev_class_dict in ret["classes"]:
            if (
                prev_class_dict["pos"][1] >= class_pos[1]
                and prev_class_dict["pos"][0] <= class_pos[0]
            ):
                class_dict["inner"] = prev_class_dict["name"]
                break
        ret["classes"].append(class_dict)
    return ret

In [119]:
for path, node in tree.filter(javalang.tree.MethodDeclaration):
    break 

In [132]:
max_line = 0
for _, child in node.filter(javalang.tree.Node):
    with contextlib.suppress(TypeError):
        max_line = max(max_line, child.position[0])

In [151]:
os.path.dirname("workdir/refs/Lang1/src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java"[
    "workdir/refs/Lang1/src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java".index((tree.package.name).replace(".", "/")):])


'org/apache/commons/lang3/math'

In [103]:
t = "testToIntStringI" 

In [423]:
content = file_utils.readFile("../workdir/refs/Lang1/src/test/java/org/apache/commons/lang3/math/NumberUtilsTest.java")
tree = parser_utils.parse(content)

In [411]:
import javalang 

parent_child = {}

for path, node in tree.filter(javalang.tree.ClassDeclaration):
    classDeclNode: javalang.tree.ClassDeclaration = node
    class_name = classDeclNode.name 
    if class_name == 'NumberUtilsTest':
        for method in classDeclNode.methods:
            if method.name == 'testToIntStringI':
                ...
                pass 
        pass 


In [412]:
node.position, get_final_line(node)

(Position(line=37, column=8), 1401)

In [426]:
node.methods[-1].position[0], get_final_line(node.methods[-1])

(1382, 1401)

In [413]:
childs = [child for _, child in node]

In [433]:
def get_method_body(file_content:str, start_lno:int, end_lno:int) -> str:
  """
  this is something that needs to be implemented
  """
  lines = file_content.split("\n")
  init_body = "\n".join(lines[start_lno-1:end_lno]) 
  cnt_close_bracket = init_body.count("}")
  cnt_open_bracket = init_body.count("{")

  n_missing_bracket = cnt_open_bracket - cnt_close_bracket
  print (n_missing_bracket)
  additional = []
  for line in lines[end_lno:]: 
    additional.append(line)
    if "}" in line:
      n_missing_bracket -= 1 # also need to update  
    if "{" in line: 
      n_missing_bracket += 1
    # check whether to end
    if n_missing_bracket == 0:
      break 
  init_body += "\n" + "\n".join(additional)
  return init_body

In [434]:
print(get_method_body(content, 1382, 1401))

1
    public void testLang381() {
        assertTrue(Double.isNaN(NumberUtils.min(1.2, 2.5, Double.NaN)));
        assertTrue(Double.isNaN(NumberUtils.max(1.2, 2.5, Double.NaN)));
        assertTrue(Float.isNaN(NumberUtils.min(1.2f, 2.5f, Float.NaN)));
        assertTrue(Float.isNaN(NumberUtils.max(1.2f, 2.5f, Float.NaN)));

        final double[] a = new double[] { 1.2, Double.NaN, 3.7, 27.0, 42.0, Double.NaN };
        assertTrue(Double.isNaN(NumberUtils.max(a)));
        assertTrue(Double.isNaN(NumberUtils.min(a)));

        final double[] b = new double[] { Double.NaN, 1.2, Double.NaN, 3.7, 27.0, 42.0, Double.NaN };
        assertTrue(Double.isNaN(NumberUtils.max(b)));
        assertTrue(Double.isNaN(NumberUtils.min(b)));

        final float[] aF = new float[] { 1.2f, Float.NaN, 3.7f, 27.0f, 42.0f, Float.NaN };
        assertTrue(Float.isNaN(NumberUtils.max(aF)));

        final float[] bF = new float[] { Float.NaN, 1.2f, Float.NaN, 3.7f, 27.0f, 42.0f, Float.NaN };

        assert

In [340]:
def get_mth_final_line(node: javalang.tree.Node) -> int:
    # traverse node and find the max line
    max_line = 0
    for path, mth in node.filter(javalang.tree.MethodDeclaration):#javalang.tree.Node):
        #with contextlib.suppress(TypeError):
        #    max_line = max(max_line, child.position[0])
        mth_last_lno = get_final_line(mth) 
        with contextlib.suppress(TypeError):
            max_line = max(max_line, mth_last_lno)
    return max_line

In [ ]:
for path, node in tree.filter(javalang.tree.MethodDeclaration):
    print (node)

In [408]:
mth_last_lnos = []
methods = []
for mth in node.methods:
    mth_last_lno = get_final_line(mth)
    mth_last_lnos.append(mth_last_lno)
    

In [409]:
max(mth_last_lnos), mth_last_lnos[-3:]

(1401, [1371, 1378, 1401])

In [387]:
node.methods[-1].position, get_final_line(node.methods[-1])

(Position(line=1382, column=12), 1404)

In [346]:
get_final_line(node.methods[-1])

1404

In [388]:
mth = node.methods[-1]

In [384]:
def get_final_line(node: javalang.tree.Node) -> int:
    # traverse node and find the max line
    max_line = 0
    for path, child in node.filter(javalang.tree.Node):
        with contextlib.suppress(TypeError):
            max_line = max(max_line, child.position[0])
    return max_line

In [389]:
mth.position, get_final_line(mth)

(Position(line=1382, column=12), 1404)

In [390]:
met = False 
next_node = None 
idx_a, idx_b = None, None
for i, (_, child) in enumerate(node):
    if met:
        next_node =child 
        idx_b = i
        break 
    if child == mth: 
        met =True 
        idx_a = i


In [391]:
next_node, idx_a, idx_b

(Annotation(element=None, name=Test), 4991, 4992)

In [394]:
get_final_line(node), get_final_line(mth)

(1404, 1404)

In [290]:
get_final_line(list(node)[4991 + 117][1])

IndexError: list index out of range

In [397]:
list(node)[-1][-1].position

Position(line=1404, column=12)

In [294]:
list(node)[4991 + 117-1][1].position

Position(line=1403, column=11)

In [310]:
list(mth)[-1][1]#.position

Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value=1)

In [313]:
len(list(mth.filter(javalang.tree.BlockStatement)))

1

In [315]:
childs = list(mth)

In [320]:
lnos = []
for _, child in childs:
    if child.position is not None:
        lnos.append(child.position[0])

In [435]:
for project in projects:
    datafile = os.path.join(datadir, f"{project}.target_latentmuts.pkl")
    if not os.path.exists(datafile):
        continue 
    all_datafile = os.path.join(datadir, f"{project}.all_latentmuts.pkl")

    all_df = pd.read_pickle(all_datafile)
    df = pd.read_pickle(datafile)
    pass 


['Lang',
 'Math',
 'Time',
 'Compress',
 'Cli',
 'Codec',
 'Closure',
 'Collections',
 'Jsoup',
 'Csv',
 'JxPath',
 'JacksonXml',
 'JacksonCore']

In [437]:
os.getcwd(), datadir

('/Users/jeongju.sohn/workdir/mutPropRepli/ext_script',
 '../output/evaluation/killable_check')